# import package

In [1]:
import pandas as pd
import numpy as np
import datetime
import json
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup

#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', None)
#设置value的显示长度为100，默认为50
pd.set_option('max_colwidth',100)

# import datasets

## about Covid-19 case

In [2]:
def formthepath():
    to_day = datetime.date.today() + datetime.timedelta(-2)
    formatted_day= to_day.strftime('%m-%d-%Y') 
    return formatted_day

doc_day = formthepath()
case_path = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{}.csv".format(doc_day)
case_df = pd.read_csv(case_path)

state = 'Florida'
case_df = case_df.query('Province_State == @state').dropna().reset_index(drop = True)
case_df = case_df.rename(columns = {'Lat':'Latitude', 'Long_':'Longitude','Admin2': 'County'})
case_df

,FIPS,County,Province_State,Country_Region,Last_Update,Latitude,Longitude,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,12001.0,Alachua,Florida,US,2020-07-07 04:34:00,29.678665,-82.359282,1636,12,0,1624.0,"Alachua, Florida, US",608.081236,0.733496
1,12003.0,Baker,Florida,US,2020-07-07 04:34:00,30.330601,-82.284675,106,4,0,102.0,"Baker, Florida, US",362.889421,3.773585
2,12005.0,Bay,Florida,US,2020-07-07 04:34:00,30.265487,-85.621226,744,5,0,739.0,"Bay, Florida, US",425.860737,0.672043
3,12007.0,Bradford,Florida,US,2020-07-07 04:34:00,29.950797,-82.166116,95,2,0,93.0,"Bradford, Florida, US",336.867487,2.105263
4,12009.0,Brevard,Florida,US,2020-07-07 04:34:00,28.294095,-80.730910,2591,19,0,2572.0,"Brevard, Florida, US",430.440142,0.733308
5,12011.0,Broward,Florida,US,2020-07-07 04:34:00,26.151847,-80.487256,21856,414,0,21442.0,"Broward, Florida, US",1119.226046,1.894217
6,12013.0,Calhoun,Florida,US,2020-07-07 04:34:00,30.406667,-85.193942,97,6,0,91.0,"Calhoun, Florida, US",687.699397,6.185567
7,12015.0,Charlotte,Florida,US,2020-07-07 04:34:00,26.901310,-81.929491,935,77,0,858.0,"Charlotte, Florida, US",494.944683,8.235294
8,12017.0,Citrus,Florida,US,2020-07-07 04:34:00,28.848043,-82.476147,399,13,0,386.0,"Citrus, Florida, US",266.609647,3.258145
9,12019.0,Clay,Florida,US,2020-07-07 04:34:00,29.983191,-81.856100,1017,34,0,983.0,"Clay, Florida, US",463.849817,3.343166


## about age

In [3]:
with urlopen('https://opendata.arcgis.com/datasets/a7887f1940b34bf5a02c6f7f27a5cb2c_0.geojson') as f:
    data = json.load(f)

#make empty dictionary and dataframe
to_make_dataframe = {}
df = pd.DataFrame(columns = data['features'][0]['properties'].keys())

#retrieve data from geojson documentation
for j in range(len(data['features'])):
    for k in data['features'][j]['properties'].keys():
            to_make_dataframe[k] = data['features'][j]['properties'][k]
    df_temp = pd.DataFrame([to_make_dataframe])
    df = pd.concat([df,df_temp],axis = 0,ignore_index=True)

#pay attention to the last row, the countyname is unknown, so I drop the last row
df.drop([67], inplace = True)
df = df.sort_values(by = 'COUNTY')

df.keys()

retrieve_list_1 = ['County_1','PUIsTotal', 'Age_0_4', 'Age_5_14', 'Age_15_24',
       'Age_25_34', 'Age_35_44', 'Age_45_54', 'Age_55_64', 'Age_65_74',
       'Age_75_84', 'Age_85plus','TPositive', 'TNegative','C_Female', 'C_Male',
       'C_SexUnkn', 'C_AllResTypes', 'C_Age_0_4', 'C_Age_5_14', 'C_Age_15_24',
       'C_Age_25_34', 'C_Age_35_44', 'C_Age_45_54', 'C_Age_55_64',
       'C_Age_65_74', 'C_Age_75_84', 'C_Age_85plus','T_total', 'T_negative', 'T_positive', 'Deaths']


retrieve_list_2 = ['County_1','TPositive', 'TNegative','C_Female', 'C_Male',
       'C_SexUnkn', 'C_AllResTypes', 'C_Age_0_4', 'C_Age_5_14', 'C_Age_15_24',
       'C_Age_25_34', 'C_Age_35_44', 'C_Age_45_54', 'C_Age_55_64',
       'C_Age_65_74', 'C_Age_75_84', 'C_Age_85plus','T_total', 'T_negative', 'T_positive', 'Deaths']


age_df = df.loc[:,retrieve_list_2].reset_index(drop = True).rename(columns = {'County_1': "County"})
age_df

,County,TPositive,TNegative,C_Female,C_Male,C_SexUnkn,C_AllResTypes,C_Age_0_4,C_Age_5_14,C_Age_15_24,C_Age_25_34,C_Age_35_44,C_Age_45_54,C_Age_55_64,C_Age_65_74,C_Age_75_84,C_Age_85plus,T_total,T_negative,T_positive,Deaths
0,Alachua,1792,14802,882,870,5,1757,14,41,614,430,188,182,146,111,42,22,47050,45247,1792,12
1,Baker,120,862,50,67,3,120,1,4,20,17,20,13,18,12,12,3,2121,2001,120,4
2,Bay,880,3012,442,401,2,845,13,18,158,193,159,138,104,60,29,8,9666,8762,880,5
3,Bradford,105,1424,71,34,0,105,1,6,13,17,16,12,9,8,12,11,2897,2790,105,2
4,Brevard,2813,18865,1437,1350,8,2795,37,76,603,596,431,397,343,179,91,59,54526,51704,2812,20
5,Broward,23783,89053,11851,11447,129,23427,397,877,3345,4875,4155,3879,3232,1643,916,454,240030,216103,23781,419
6,Calhoun,106,476,71,35,0,106,0,0,14,11,8,8,12,11,21,21,1027,918,106,6
7,Charlotte,975,4934,540,420,11,971,12,14,91,106,89,121,166,144,111,119,13990,13009,974,77
8,Citrus,457,3956,220,234,3,457,6,15,55,64,47,65,97,58,34,16,11396,10935,457,13
9,Clay,1158,7819,625,524,5,1154,7,31,160,194,176,197,159,123,65,45,19964,18793,1158,34


## about medical resources

In [4]:
ven_path = 'https://raw.githubusercontent.com/Archerontheway/Summer_C_project/master/Data%20Source/Raw_dataset/ventilator_data.csv'
doc_path = 'https://raw.githubusercontent.com/Archerontheway/Summer_C_project/master/Data%20Source/medical_resource_dataset/medical_resource.csv'

ven_df = pd.read_csv(ven_path)
doc_df = pd.read_csv(doc_path)

medical_df = pd.merge(doc_df,ven_df,how = 'left', on = 'County')
medical_df

,County,Clinician_Count_MD,Clinician_Count_DO,Nurse_practitioner_Count,Available beds,Bed Census,Total Staffed Bed Capacity,Est Available Ventilators,County ICU Beds,County Staffed Beds,Hospitalized,ICU Cases,Latest COVID-19 Cases,Percent Remaining Vent Capacity,Ventilator Capacity
0,Alachua,2225,74,600,267.0,1397.0,1664.0,110.0,387,"2,263",13.0,4.0,716.0,0.990692,430
1,Baker,17,2,17,95.0,530.0,625.0,3.0,4,81,1.0,1.0,41.0,0.750000,4
2,Bay,347,27,164,109.0,414.0,523.0,49.0,77,676,4.0,1.0,190.0,0.986711,75
3,Bradford,17,1,18,0.0,0.0,0.0,3.0,4,25,2.0,1.0,61.0,0.750000,4
4,Brevard,1239,118,421,410.0,1153.0,1563.0,125.0,288,"1,867",15.0,5.0,850.0,0.983607,305
5,Broward,4498,678,1357,1707.0,4172.0,5879.0,293.0,790,"5,791",191.0,56.0,11155.0,0.935223,864
6,Calhoun,7,1,10,3.0,7.0,10.0,2.0,3,15,2.0,1.0,69.0,0.636364,3
7,Charlotte,304,59,116,227.0,507.0,734.0,51.0,51,595,11.0,3.0,587.0,0.934426,46
8,Citrus,201,32,86,97.0,217.0,314.0,30.0,40,332,4.0,1.0,189.0,0.972222,36
9,Clay,308,41,158,46.0,447.0,493.0,55.0,42,459,9.0,3.0,494.0,0.904762,32


## about population

In [5]:
pop_path = '/Users/chenhaoyi/JupyterNotebook/SummerProject/data/population_data/pop_data.xlsx'
df_pop_obse = pd.read_excel(pop_path)

pop_df = pd.read_excel(pop_path)[['County','Population','land area(sq mi)', 'population_density']]
obse_df = pd.read_excel(pop_path)[['County', 'overweight or obese(%)',' underweight(%)']]
pop_df

,County,Population,land area(sq mi),population_density
0,Alachua,263148,875.02,300.733698
1,Baker,27785,585.23,47.477060
2,Bay,182482,758.46,240.595417
3,Bradford,26979,293.96,91.777793
4,Brevard,576808,1015.66,567.914460
5,Broward,1909151,1209.78,1578.097671
6,Calhoun,14444,567.33,25.459609
7,Charlotte,176954,680.28,260.119363
8,Citrus,143087,581.70,245.980746
9,Clay,207291,604.36,342.992587


## about economics

In [6]:
eco_path = '/Users/chenhaoyi/JupyterNotebook/SummerProject/data/economic_dataset/economic_data.xlsx'
eco_df = pd.read_excel(eco_path)

eco_df

,County,GDP,income,FPLi,cases,death,lethality,perin,logG,Logperin,Logl,population,pergdp
0,Alachua,12447381,11983398,97.45,551,10,0.018149,44390,7.095078,4.647285,-1.741152,267306,46.566037
1,Baker,595832,916242,96.45,30,4,0.133333,32313,5.775124,4.509377,-0.875061,28249,21.092145
2,Bay,7606775,8010201,95.83,153,4,0.026144,43231,6.881201,4.635795,-1.582631,167283,45.472493
3,Bradford,659865,930593,95.83,52,2,0.038462,33557,5.819455,4.525783,-1.414973,28682,23.006241
4,Brevard,23437467,27112075,98.36,565,15,0.026549,45425,7.369911,4.657295,-1.575957,594469,39.425886
5,Broward,96874849,98087689,102.04,9086,358,0.039401,50269,7.986211,4.701300,-1.404490,1919644,50.465008
6,Calhoun,250104,391997,91.43,66,5,0.075758,26873,5.398121,4.429316,-1.120574,14067,17.779484
7,Charlotte,4670656,7689186,98.71,537,73,0.135940,41564,6.669378,4.618717,-0.866651,181770,25.695417
8,Citrus,3511018,5609441,92.98,150,12,0.080000,37920,6.545433,4.578868,-1.096910,147744,23.764200
9,Clay,4508532,9271351,98.38,440,32,0.072727,42909,6.654035,4.632548,-1.138303,215246,20.945950


## about mobility

In [7]:
data_path = '/Users/chenhaoyi/JupyterNotebook/SummerProject/data/Mobility_data/applemobilitytrends_2020_06_29.xlsx'
mobility_data = pd.read_excel(data_path)
county_data = mobility_data[mobility_data['county'].str.contains('County')].reset_index(drop = True)

mobility_df = county_data.drop(columns = ['geo_type','sub-region','country'],axis = 1)
mobility_df['county'] = mobility_df['county'].apply(lambda x: x[:-7])
mobility_df

,county,transportation_type,2020-01-13,2020-01-14,2020-01-15,2020-01-16,2020-01-17,2020-01-18,2020-01-19,2020-01-20,2020-01-21,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31,2020-02-01,2020-02-02,2020-02-03,2020-02-04,2020-02-05,2020-02-06,2020-02-07,2020-02-08,2020-02-09,2020-02-10,2020-02-11,2020-02-12,2020-02-13,2020-02-14,2020-02-15,2020-02-16,2020-02-17,2020-02-18,2020-02-19,2020-02-20,2020-02-21,2020-02-22,2020-02-23,2020-02-24,2020-02-25,2020-02-26,2020-02-27,2020-02-28,2020-02-29,2020-03-01,2020-03-02,2020-03-03,2020-03-04,2020-03-05,2020-03-06,2020-03-07,2020-03-08,2020-03-09,2020-03-10,2020-03-11,2020-03-12,2020-03-13,2020-03-14,2020-03-15,2020-03-16,2020-03-17,2020-03-18,2020-03-19,2020-03-20,2020-03-21,2020-03-22,2020-03-23,2020-03-24,2020-03-25,2020-03-26,2020-03-27,2020-03-28,2020-03-29,2020-03-30,2020-03-31,2020-04-01,2020-04-02,2020-04-03,2020-04-04,2020-04-05,2020-04-06,2020-04-07,2020-04-08,2020-04-09,2020-04-10,2020-04-11,2020-04-12,2020-04-13,2020-04-14,2020-04-15,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-20,2020-04-21,2020-04-22,2020-04-23,2020-04-24,2020-04-25,2020-04-26,2020-04-27,2020-04-28,2020-04-29,2020-04-30,2020-05-01,2020-05-02,2020-05-03,2020-05-04,2020-05-05,2020-05-06,2020-05-07,2020-05-08,2020-05-09,2020-05-10,2020-05-11,2020-05-12,2020-05-13,2020-05-14,2020-05-15,2020-05-16,2020-05-17,2020-05-18,2020-05-19,2020-05-20,2020-05-21,2020-05-22,2020-05-23,2020-05-24,2020-05-25,2020-05-26,2020-05-27,2020-05-28,2020-05-29,2020-05-30,2020-05-31,2020-06-01,2020-06-02,2020-06-03,2020-06-04,2020-06-05,2020-06-06,2020-06-07,2020-06-08,2020-06-09,2020-06-10,2020-06-11,2020-06-12,2020-06-13,2020-06-14,2020-06-15,2020-06-16,2020-06-17,2020-06-18,2020-06-19,2020-06-20,2020-06-21,2020-06-22,2020-06-23,2020-06-24,2020-06-25,2020-06-26,2020-06-27,2020-06-28,2020-06-29
0,Alachua,driving,100,102.21,111.36,116.55,141.97,130.21,92.69,104.06,101.20,103.93,106.16,131.05,120.30,91.92,97.30,102.00,99.21,110.02,144.46,129.98,93.15,97.86,100.93,105.94,108.80,135.37,128.78,99.12,100.18,105.30,108.33,127.00,157.51,143.37,108.81,107.82,107.65,110.22,116.62,140.65,135.36,102.54,100.90,103.86,119.22,124.22,157.90,117.31,86.23,89.77,89.19,91.26,94.83,114.68,119.74,99.42,108.03,110.97,113.35,119.84,140.00,119.29,86.35,91.64,85.15,79.61,76.55,81.44,67.81,50.88,55.82,50.67,52.95,52.62,57.60,52.28,41.10,49.14,49.81,51.99,51.90,53.91,45.36,34.70,46.15,47.72,47.88,50.62,56.55,50.89,33.68,50.68,48.61,51.46,56.66,63.26,54.34,47.48,54.18,57.14,59.33,61.13,64.23,62.90,49.31,60.22,62.65,65.62,66.78,85.60,72.56,59.45,68.54,73.79,73.60,76.47,86.91,76.87,55.54,NaN,NaN,80.97,85.03,98.57,91.66,76.43,79.95,86.47,88.27,97.84,110.89,100.32,81.67,75.80,88.85,91.17,94.49,111.02,106.77,86.88,92.84,92.63,92.19,96.90,117.39,106.39,79.76,95.01,99.14,101.95,103.45,123.29,123.28,95.84,101.77,102.80,104.84,109.38,130.16,121.64,87.92,105.58,104.22,107.49,115.48,127.04,122.16,95.71,104.81
1,Baker,driving,100,106.73,104.79,115.21,160.07,136.27,107.89,124.29,100.76,102.84,109.70,133.63,136.93,101.12,102.71,99.64,103.96,111.82,147.92,132.97,99.77,112.41,101.98,107.03,113.33,147.85,122.24,115.41,110.07,109.54,116.57,134.95,175.64,166.24,131.82,127.52,114.72,117.29,126.34,156.53,155.38,120.43,114.72,111.98,111.02,156.80,166.07,160.59,132.54,116.30,113.60,115.38,133.93,178.05,172.94,139.74,129.83,126.47,132.34,154.75,197.72,176.77,140.20,132.57,109.57,114.98,117.49,128.91,114.88,97.23,92.57,86.37,88.88,91.75,108.25,99.60,79.83,98.91,83.86,76.30,90.56,87.62,65.68,57.95,76.53,74.52,68.45,79.50,89.21,86.17,58.02,73.89,69.80,73.76,88.84,97.26,89.74,78.98,84.32,88.68,94.75,97.59,104.79,97.19,89.70,99.27,97.00,97.23,100.30,126.20,127.52,104.59,114.06,111.02,112.18,125.68,147.46,127.26,101.22,NaN,NaN,126.67,128.28,160.17,166.83,138.38,128.22,134.36,125.15,157.03,201.16,174.29,144.36,154.62,136.47,126.67,143.43,169.24,178.42,149.74,153.07,129.80,136.93,141.19,172.81,190.79,137.39,156.17,131.62,154.6

## Extra data from open health Florida

In [8]:
with urlopen('https://opendata.arcgis.com/datasets/a7887f1940b34bf5a02c6f7f27a5cb2c_0.geojson') as f:
    data = json.load(f)

#make empty dictionary and dataframe
to_make_dataframe = {}
df = pd.DataFrame(columns = data['features'][0]['properties'].keys())

#retrieve data from geojson documentation
for j in range(len(data['features'])):
    for k in data['features'][j]['properties'].keys():
            to_make_dataframe[k] = data['features'][j]['properties'][k]
    df_temp = pd.DataFrame([to_make_dataframe])
    df = pd.concat([df,df_temp],axis = 0,ignore_index=True)

#pay attention to the last row, the countyname is unknown, so I drop the last row
df.drop([67], inplace = True)
df = df.sort_values(by = 'COUNTY')
#df.keys()


retrieve_list = ['County_1','C_RaceWhite', 'C_RaceBlack',
       'C_RaceOther', 'C_RaceUnknown', 'C_HispanicYES', 'C_HispanicNO',
       'C_HispanicUnk', 'C_HospYes_Res',
       'C_HospYes_NonRes', 'C_NonResDeaths', 'C_FLResDeaths', 'CasesAll',
       'C_Men', 'C_Women', 'C_FLRes', 'C_NotFLRes', 'C_FLResOut', 'T_NegRes',
       'T_NegNotFLRes', 'T_total', 'T_negative', 'T_positive', 'Deaths']

opendata_df = df.loc[:,retrieve_list].reset_index(drop = True).rename(columns = {'County_1': "County"})

for i in range(len(opendata_df.keys())):
               if i == 0:
                   pass
               else:
                   opendata_df.iloc[:,i] = opendata_df.iloc[:,i].astype('float')
               
                            
opendata_df

,County,C_RaceWhite,C_RaceBlack,C_RaceOther,C_RaceUnknown,C_HispanicYES,C_HispanicNO,C_HispanicUnk,C_HospYes_Res,C_HospYes_NonRes,C_NonResDeaths,C_FLResDeaths,CasesAll,C_Men,C_Women,C_FLRes,C_NotFLRes,C_FLResOut,T_NegRes,T_NegNotFLRes,T_total,T_negative,T_positive,Deaths
0,Alachua,607.0,356.0,269.0,525.0,277.0,916.0,564.0,96.0,5.0,0.0,12.0,1792.0,890.0,897.0,1757.0,35.0,0.0,44997.0,250.0,47050.0,45247.0,1792.0,12.0
1,Baker,82.0,20.0,5.0,13.0,2.0,99.0,19.0,20.0,0.0,0.0,4.0,120.0,67.0,50.0,120.0,0.0,0.0,1941.0,60.0,2121.0,2001.0,120.0,4.0
2,Bay,542.0,109.0,46.0,148.0,142.0,520.0,183.0,39.0,2.0,2.0,5.0,880.0,422.0,456.0,845.0,35.0,0.0,8741.0,21.0,9666.0,8762.0,880.0,5.0
3,Bradford,72.0,26.0,3.0,4.0,6.0,93.0,6.0,14.0,0.0,0.0,2.0,105.0,34.0,71.0,105.0,0.0,0.0,2788.0,2.0,2897.0,2790.0,105.0,2.0
4,Brevard,1567.0,176.0,187.0,865.0,447.0,1467.0,881.0,143.0,3.0,0.0,20.0,2812.0,1357.0,1447.0,2795.0,17.0,0.0,51698.0,6.0,54526.0,51704.0,2812.0,20.0
5,Broward,9841.0,6666.0,1805.0,5115.0,7961.0,10596.0,4870.0,2354.0,60.0,22.0,419.0,23781.0,11678.0,11974.0,23427.0,354.0,0.0,214843.0,1260.0,240030.0,216103.0,23781.0,419.0
6,Calhoun,75.0,14.0,7.0,10.0,9.0,81.0,16.0,12.0,0.0,0.0,6.0,106.0,35.0,71.0,106.0,0.0,0.0,918.0,0.0,1027.0,918.0,106.0,6.0
7,Charlotte,730.0,71.0,20.0,150.0,67.0,721.0,183.0,158.0,1.0,0.0,77.0,974.0,421.0,541.0,971.0,3.0,0.0,12993.0,16.0,13990.0,13009.0,974.0,77.0
8,Citrus,342.0,25.0,28.0,62.0,35.0,350.0,72.0,47.0,0.0,0.0,13.0,457.0,234.0,220.0,457.0,0.0,0.0,10929.0,6.0,11396.0,10935.0,457.0,13.0
9,Clay,505.0,198.0,85.0,366.0,94.0,652.0,408.0,112.0,0.0,0.0,34.0,1158.0,526.0,627.0,1154.0,4.0,0.0,18775.0,18.0,19964.0,18793.0,1158.0,34.0


## the combined dataset

In [17]:
combined_age_medical_df = pd.merge(age_df, medical_df,how = 'inner', on = 'County')
combined_eco_pop_df =  pd.merge(pop_df, eco_df,how = 'inner', on = 'County')

combined_all_df = pd.merge(combined_eco_pop_df, combined_age_medical_df,how = 'inner', on = 'County')
#combined_all_df.to_csv('/Users/chenhaoyi/JupyterNotebook/SummerProject/data/combined_all_df.csv')
combined_all_df

,County,Population,land area(sq mi),population_density,GDP,income,FPLi,cases,death,lethality,perin,logG,Logperin,Logl,population,pergdp,TPositive,TNegative,C_Female,C_Male,C_SexUnkn,C_AllResTypes,C_Age_0_4,C_Age_5_14,C_Age_15_24,C_Age_25_34,C_Age_35_44,C_Age_45_54,C_Age_55_64,C_Age_65_74,C_Age_75_84,C_Age_85plus,T_total,T_negative,T_positive,Deaths,Clinician_Count_MD,Clinician_Count_DO,Nurse_practitioner_Count,Available beds,Bed Census,Total Staffed Bed Capacity,Est Available Ventilators,County ICU Beds,County Staffed Beds,Hospitalized,ICU Cases,Latest COVID-19 Cases,Percent Remaining Vent Capacity,Ventilator Capacity
0,Alachua,263148,875.02,300.733698,12447381,11983398,97.45,551,10,0.018149,44390,7.095078,4.647285,-1.741152,267306,46.566037,1792,14802,882,870,5,1757,14,41,614,430,188,182,146,111,42,22,47050,45247,1792,12,2225,74,600,267.0,1397.0,1664.0,110.0,387,"2,263",13.0,4.0,716.0,0.990692,430
1,Baker,27785,585.23,47.477060,595832,916242,96.45,30,4,0.133333,32313,5.775124,4.509377,-0.875061,28249,21.092145,120,862,50,67,3,120,1,4,20,17,20,13,18,12,12,3,2121,2001,120,4,17,2,17,95.0,530.0,625.0,3.0,4,81,1.0,1.0,41.0,0.750000,4
2,Bay,182482,758.46,240.595417,7606775,8010201,95.83,153,4,0.026144,43231,6.881201,4.635795,-1.582631,167283,45.472493,880,3012,442,401,2,845,13,18,158,193,159,138,104,60,29,8,9666,8762,880,5,347,27,164,109.0,414.0,523.0,49.0,77,676,4.0,1.0,190.0,0.986711,75
3,Bradford,26979,293.96,91.777793,659865,930593,95.83,52,2,0.038462,33557,5.819455,4.525783,-1.414973,28682,23.006241,105,1424,71,34,0,105,1,6,13,17,16,12,9,8,12,11,2897,2790,105,2,17,1,18,0.0,0.0,0.0,3.0,4,25,2.0,1.0,61.0,0.750000,4
4,Brevard,576808,1015.66,567.914460,23437467,27112075,98.36,565,15,0.026549,45425,7.369911,4.657295,-1.575957,594469,39.425886,2813,18865,1437,1350,8,2795,37,76,603,596,431,397,343,179,91,59,54526,51704,2812,20,1239,118,421,410.0,1153.0,1563.0,125.0,288,"1,867",15.0,5.0,850.0,0.983607,305
5,Broward,1909151,1209.78,1578.097671,96874849,98087689,102.04,9086,358,0.039401,50269,7.986211,4.701300,-1.404490,1919644,50.465008,23783,89053,11851,11447,129,23427,397,877,3345,4875,4155,3879,3232,1643,916,454,240030,216103,23781,419,4498,678,1357,1707.0,4172.0,5879.0,293.0,790,"5,791",191.0,56.0,11155.0,0.935223,864
6,Calhoun,14444,567.33,25.459609,250104,391997,91.43,66,5,0.075758,26873,5.398121,4.429316,-1.120574,14067,17.779484,106,476,71,35,0,106,0,0,14,11,8,8,12,11,21,21,1027,918,106,6,7,1,10,3.0,7.0,10.0,2.0,3,15,2.0,1.0,69.0,0.636364,3
7,Charlotte,176954,680.28,260.119363,4670656,7689186,98.71,537,73,0.135940,41564,6.669378,4.618717,-0.866651,181770,25.695417,975,4934,540,420,11,971,12,14,91,106,89,121,166,144,111,119,13990,13009,974,77,304,59,116,227.0,507.0,734.0,51.0,51,595,11.0,3.0,587.0,0.934426,46
8,Citrus,143087,581.70,245.980746,3511018,5609441,92.98,150,12,0.080000,37920,6.545433,4.578868,-1.096910,147744,23.764200,457,3956,220,234,3,457,6,15,55,64,47,65,97,58,34,16,11396,10935,457,13,201,32,86,97.0,217.0,314.0,30.0,40,332,4.0,1.0,189.0,0.972222,36
9,Clay,207291,604.36,342.992587,4508532,9271351,98.38,440,32,0.072727,42909,6.654035,4.632548,-1.138303,215246,20.945950,1158,7819,625,524,5,1154,7,31,160,194,176,197,159,123,65,45,19964,18793,1158,34,308,41,158,46.0,447.0,493.0,55.0,42,459,9.0,3.0,494.0,0.904762,32


## create new variable

In [10]:
with urlopen('https://opendata.arcgis.com/datasets/a7887f1940b34bf5a02c6f7f27a5cb2c_0.geojson') as f:
    data = json.load(f)

#make empty dictionary and dataframe
to_make_dataframe = {}
df = pd.DataFrame(columns = data['features'][0]['properties'].keys())

#retrieve data from geojson documentation
for j in range(len(data['features'])):
    for k in data['features'][j]['properties'].keys():
            to_make_dataframe[k] = data['features'][j]['properties'][k]
    df_temp = pd.DataFrame([to_make_dataframe])
    df = pd.concat([df,df_temp],axis = 0,ignore_index=True)

#pay attention to the last row, the countyname is unknown, so I drop the last row
df.drop([67], inplace = True)
df = df.sort_values(by = 'COUNTY')

df.keys()

retrieve_list = ['County_1','TPositive', 'TNegative','C_Female', 'C_Male',
       'C_SexUnkn', 'C_AllResTypes', 'C_Age_0_4', 'C_Age_5_14', 'C_Age_15_24',
       'C_Age_25_34', 'C_Age_35_44', 'C_Age_45_54', 'C_Age_55_64',
       'C_Age_65_74', 'C_Age_75_84', 'C_Age_85plus','T_total', 'T_negative', 'T_positive', 'Deaths']

age_add_variable = df.loc[:,retrieve_list].reset_index(drop = True).rename(columns = {'County_1': "County"})
age_add_variable['lethality_ratio'] = (age_add_variable['Deaths'] / age_add_variable['T_positive'])*100
age_add_variable['C_plus_65'] = age_add_variable['C_Age_65_74'] + age_add_variable['C_Age_75_84'] + age_add_variable['C_Age_85plus']
age_add_variable['C_plus_65_ratio'] = (age_add_variable['C_plus_65'] / age_add_variable['T_positive']) * 100

for i in range(len(age_add_variable.keys())):
               if i == 0:
                   pass
               else:
                   age_add_variable.iloc[:,i] = age_add_variable.iloc[:,i].astype('float')
               
                            
age_add_variable

,County,TPositive,TNegative,C_Female,C_Male,C_SexUnkn,C_AllResTypes,C_Age_0_4,C_Age_5_14,C_Age_15_24,C_Age_25_34,C_Age_35_44,C_Age_45_54,C_Age_55_64,C_Age_65_74,C_Age_75_84,C_Age_85plus,T_total,T_negative,T_positive,Deaths,lethality_ratio,C_plus_65,C_plus_65_ratio
0,Alachua,1792.0,14802.0,882.0,870.0,5.0,1757.0,14.0,41.0,614.0,430.0,188.0,182.0,146.0,111.0,42.0,22.0,47050.0,45247.0,1792.0,12.0,0.669643,175.0,9.765625
1,Baker,120.0,862.0,50.0,67.0,3.0,120.0,1.0,4.0,20.0,17.0,20.0,13.0,18.0,12.0,12.0,3.0,2121.0,2001.0,120.0,4.0,3.333333,27.0,22.500000
2,Bay,880.0,3012.0,442.0,401.0,2.0,845.0,13.0,18.0,158.0,193.0,159.0,138.0,104.0,60.0,29.0,8.0,9666.0,8762.0,880.0,5.0,0.568182,97.0,11.022727
3,Bradford,105.0,1424.0,71.0,34.0,0.0,105.0,1.0,6.0,13.0,17.0,16.0,12.0,9.0,8.0,12.0,11.0,2897.0,2790.0,105.0,2.0,1.904762,31.0,29.523810
4,Brevard,2813.0,18865.0,1437.0,1350.0,8.0,2795.0,37.0,76.0,603.0,596.0,431.0,397.0,343.0,179.0,91.0,59.0,54526.0,51704.0,2812.0,20.0,0.711238,329.0,11.699858
5,Broward,23783.0,89053.0,11851.0,11447.0,129.0,23427.0,397.0,877.0,3345.0,4875.0,4155.0,3879.0,3232.0,1643.0,916.0,454.0,240030.0,216103.0,23781.0,419.0,1.761911,3013.0,12.669778
6,Calhoun,106.0,476.0,71.0,35.0,0.0,106.0,0.0,0.0,14.0,11.0,8.0,8.0,12.0,11.0,21.0,21.0,1027.0,918.0,106.0,6.0,5.660377,53.0,50.000000
7,Charlotte,975.0,4934.0,540.0,420.0,11.0,971.0,12.0,14.0,91.0,106.0,89.0,121.0,166.0,144.0,111.0,119.0,13990.0,13009.0,974.0,77.0,7.905544,374.0,38.398357
8,Citrus,457.0,3956.0,220.0,234.0,3.0,457.0,6.0,15.0,55.0,64.0,47.0,65.0,97.0,58.0,34.0,16.0,11396.0,10935.0,457.0,13.0,2.844639,108.0,23.632385
9,Clay,1158.0,7819.0,625.0,524.0,5.0,1154.0,7.0,31.0,160.0,194.0,176.0,197.0,159.0,123.0,65.0,45.0,19964.0,18793.0,1158.0,34.0,2.936097,233.0,20.120898


## extra data specific for age

In [11]:
extra_path = '/Users/chenhaoyi/JupyterNotebook/SummerProject/data/age_dataset/Provisional_COVID-19_Death_Counts_by_Sex__Age__and_State.csv'
extra_df = pd.read_csv(extra_path)

state = 'Florida'
extra_FL_df = extra_df.query('State == @state').loc[:,['Sex','Age group','COVID-19 Deaths','Total Deaths','Pneumonia Deaths','Pneumonia and COVID-19 Deaths','Influenza Deaths','Pneumonia, Influenza, or COVID-19 Deaths']].reset_index(drop = True)
extra_FL_df

,Sex,Age group,COVID-19 Deaths,Total Deaths,Pneumonia Deaths,Pneumonia and COVID-19 Deaths,Influenza Deaths,"Pneumonia, Influenza, or COVID-19 Deaths"
0,Male,Under 1 year,0.0,275.0,NaN,0.0,0.0,NaN
1,Male,1-4 years,0.0,56.0,NaN,0.0,NaN,NaN
2,Male,5-14 years,0.0,86.0,NaN,0.0,NaN,NaN
3,Male,15-24 years,0.0,645.0,10.0,0.0,NaN,13.0
4,Male,25-34 years,10.0,1340.0,30.0,NaN,NaN,40.0
5,Male,35-44 years,40.0,1733.0,70.0,19.0,NaN,97.0
6,Male,45-54 years,85.0,2971.0,177.0,47.0,20.0,233.0
7,Male,55-64 years,192.0,6914.0,475.0,114.0,31.0,584.0
8,Male,65-74 years,379.0,10529.0,950.0,203.0,36.0,1162.0
9,Male,75-84 years,502.0,12656.0,1236.0,270.0,34.0,1501.0
